## Diagnosis

While the purpose of Flaredown for it's users isn't about diagnosis, we would be remiss if we ignored the fact that we have a very solid training set for a system to perform diagnosis.  We will reshape the data so that our features are one-hotted symptoms, and we will predict on condition.

There are a lot of algorithms out there that take a list of self reported symptoms and attempt a diagnosis.  But the depth and breadth of the ever-growing Flaredown data may provide new oppertunities for this task, especially since in the future Flaredown may collect any number of additional variables.

I do recommend giving this some time before it's used.  At this time (Aug 22, 2016) the data only describes about 900 conditions.  For a diagnosis engine to be useful it is likely to require a huge breadth of conditions.